# 🚀 Fluxo Completo de MLOps - Preparação para Entrevista Loft

Este notebook cobre o **ciclo completo de deploy e monitoramento de modelos em produção** - exatamente o diferencial da vaga!

## 📋 O que você vai aprender:
1. **Pré-processamento** - Preparar dados para ML
2. **Treinamento** - Treinar e comparar modelos
3. **Tracking com MLflow** - Rastrear experimentos
4. **Registry** - Versionar modelos
5. **Scoring em Batch** - Predições em lote
6. **Pipeline Vertex AI** - Automação completa

---

## 🔧 Instalação das Dependências

Execute esta célula para instalar tudo que precisa:

In [ ]:
!pip install pandas numpy scikit-learn xgboost mlflow google-cloud-storage google-cloud-aiplatform kfp

## 📊 Parte 1: Entendendo os Dados

Primeiro, vamos carregar e explorar os dados. Isso é fundamental para qualquer projeto de ML.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# Configurações GCP
PROJECT_ID = "mlops-484912"
BUCKET_NAME = "meu-bucket-29061999"

# Carrega dados (local ou GCS)
# Opção 1: Local
df = pd.read_csv("../data/base_clientes_inadimplencia.csv")

# Opção 2: Do GCS (descomente se os dados estiverem no bucket)
# df = pd.read_csv(f"gs://{BUCKET_NAME}/data/base_clientes_inadimplencia.csv")

print(f"Shape: {df.shape}")
print(f"\nColunas: {list(df.columns)}")
df.head()

In [ ]:
# Análise da variável target
print("Distribuição do Target (Status_Pagamento):")
print(df["Status_Pagamento"].value_counts())
print(f"\n% Inadimplentes: {(df['Status_Pagamento'] == 'Inadimplente').mean()*100:.2f}%")

## 🔄 Parte 2: Pré-processamento

Esta etapa é CRUCIAL em qualquer projeto de ML. Um bom pré-processamento pode melhorar muito a performance do modelo.

**O que fazemos:**
- Tratar valores nulos
- Criar features (engenharia de features)
- Codificar variáveis categóricas
- Normalizar dados

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

def preprocessar_dados(df):
    """
    Pipeline de pré-processamento.
    
    DICA PARA ENTREVISTA: Explique cada decisão!
    """
    df = df.copy()
    
    # 1. Tratar valores nulos
    # Numéricos: mediana (robusta a outliers)
    for col in df.select_dtypes(include=[np.number]).columns:
        df[col] = df[col].fillna(df[col].median())
    
    # Categóricos: valor mais frequente ou 'desconhecido'
    for col in df.select_dtypes(include=[object]).columns:
        df[col] = df[col].fillna('desconhecido')
    
    # 2. Feature Engineering: Calcular idade
    if "Data_Nascimento" in df.columns:
        df["Data_Nascimento"] = pd.to_datetime(df["Data_Nascimento"], errors='coerce')
        ref = datetime.today()
        df["Idade"] = ref.year - df["Data_Nascimento"].dt.year
    
    # 3. Remover colunas que não ajudam na predição
    drop_cols = ["Telefone", "Nome", "Email", "Data_Nascimento", 
                 "Data_Contratacao", "Data_Vencimento_Fatura", 
                 "Data_Ingestao", "Data_Atualizacao"]
    df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')
    
    # 4. Codificar target
    if "Status_Pagamento" in df.columns:
        le = LabelEncoder()
        df["Status_Pagamento"] = le.fit_transform(df["Status_Pagamento"])
    
    # 5. One-hot encoding para categóricas
    cat_cols = df.select_dtypes(include=['object']).columns
    df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
    
    return df

# Aplica pré-processamento
df_processed = preprocessar_dados(df)
print(f"Shape após processamento: {df_processed.shape}")
df_processed.head()

## 🤖 Parte 3: Treinamento com MLflow Tracking

**MLflow** é a ferramenta padrão de mercado para rastrear experimentos de ML.

**Por que usar MLflow?**
- Rastreia parâmetros, métricas e artefatos
- Compara diferentes runs/modelos
- Versiona modelos
- Facilita reprodutibilidade

**ISSO É O QUE A LOFT VAI PERGUNTAR!**

In [ ]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Configura MLflow
# Opção 1: Local (para desenvolvimento)
mlflow.set_tracking_uri("mlruns")  # Salva localmente

# Opção 2: GCS (para produção - descomente)
# mlflow.set_tracking_uri(f"gs://{BUCKET_NAME}/mlflow")

# Cria experimento
mlflow.set_experiment("inadimplencia-experimentos")

print("MLflow configurado!")

In [ ]:
# Prepara dados para treino
target = "Status_Pagamento"
X = df_processed.drop(columns=[target, "ID_Cliente"], errors='ignore')
y = df_processed[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Treino: {X_train.shape}")
print(f"Teste: {X_test.shape}")

In [ ]:
def treinar_e_logar(model, model_name, X_train, X_test, y_train, y_test):
    """
    Treina modelo e registra tudo no MLflow.
    
    IMPORTANTE PARA ENTREVISTA:
    - Explique o que cada métrica significa
    - Por que escolheu essas métricas?
    """
    with mlflow.start_run(run_name=model_name):
        # 1. Log dos parâmetros
        mlflow.log_param("model_type", model_name)
        mlflow.log_param("train_size", len(X_train))
        mlflow.log_param("test_size", len(X_test))
        mlflow.log_param("n_features", X_train.shape[1])
        
        # 2. Treina
        model.fit(X_train, y_train)
        
        # 3. Predições
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]
        
        # 4. Calcula métricas
        metrics = {
            "accuracy": accuracy_score(y_test, y_pred),
            "f1_score": f1_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred),
            "recall": recall_score(y_test, y_pred),
            "roc_auc": roc_auc_score(y_test, y_proba),
        }
        
        # 5. Log das métricas
        for k, v in metrics.items():
            mlflow.log_metric(k, v)
            print(f"{k}: {v:.4f}")
        
        # 6. Log do modelo
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")
        
        # 7. Registra modelo no Model Registry
        mlflow.register_model(
            f"runs:/{mlflow.active_run().info.run_id}/model",
            model_name
        )
        
        print(f"\n✅ Modelo {model_name} registrado com sucesso!")
        return model, metrics

In [ ]:
# Treina XGBoost
print("="*50)
print("TREINANDO XGBOOST")
print("="*50)

xgb_model = XGBClassifier(
    objective="binary:logistic",
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=42
)

xgb_model, xgb_metrics = treinar_e_logar(
    xgb_model, "XGBoost-Inadimplencia", 
    X_train, X_test, y_train, y_test
)

In [ ]:
# Treina Random Forest
print("="*50)
print("TREINANDO RANDOM FOREST")
print("="*50)

rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)

rf_model, rf_metrics = treinar_e_logar(
    rf_model, "RandomForest-Inadimplencia",
    X_train, X_test, y_train, y_test
)

## 📈 Parte 4: Comparando Modelos

Uma das perguntas clássicas de entrevista: **"Como você escolhe o melhor modelo?"**

In [ ]:
# Compara métricas
comparison = pd.DataFrame({
    "XGBoost": xgb_metrics,
    "Random Forest": rf_metrics
}).T

print("\n📊 COMPARAÇÃO DOS MODELOS:")
print("="*50)
print(comparison.round(4))

# Qual é o melhor?
best_model = "XGBoost" if xgb_metrics["f1_score"] > rf_metrics["f1_score"] else "Random Forest"
print(f"\n🏆 Melhor modelo (por F1-Score): {best_model}")

## 🚀 Parte 5: Scoring em Batch (Produção)

Agora vamos simular como funciona em **produção**: carregar modelo do registry e fazer predições em novos dados.

**Este é o deploy de modelo!**

In [ ]:
def scoring_batch(model_name, model_version, dados_novos):
    """
    Faz scoring em batch - padrão de produção.
    
    Em produção real:
    1. Dados vêm de um bucket (GCS/S3)
    2. Modelo é carregado do registry
    3. Predições são salvas no bucket
    4. Alertas são disparados se necessário
    """
    # Carrega modelo do registry
    model_uri = f"models:/{model_name}/{model_version}"
    print(f"Carregando modelo: {model_uri}")
    
    model = mlflow.pyfunc.load_model(model_uri)
    
    # Faz predições
    predictions = model.predict(dados_novos)
    
    # Monta resultado
    resultado = dados_novos.copy()
    resultado["prediction"] = predictions
    
    return resultado

# Simula novos dados (usa o teste como exemplo)
print("\n🔮 EXECUTANDO SCORING EM BATCH")
print("="*50)

resultado = scoring_batch("XGBoost-Inadimplencia", "1", X_test)
print(f"\nPredições geradas: {len(resultado)}")
print(f"Inadimplentes previstos: {resultado['prediction'].sum()}")
resultado.head()

## ☁️ Parte 6: Pipeline no Vertex AI (Produção GCP)

**IMPORTANTE:** Esta parte precisa rodar em uma máquina com acesso ao GCP!

Você pode:
1. Usar o **Cloud Shell** do GCP (gratuito)
2. Usar **Vertex AI Workbench** (notebook gerenciado)
3. Configurar gcloud na sua máquina local

O código abaixo mostra como criar e executar um pipeline completo.

In [ ]:
# CÓDIGO PARA RODAR NO GCP
# ==========================
# Copie este código para o Cloud Shell ou Vertex AI Workbench

PIPELINE_CODE = '''
from google.cloud import aiplatform
from kfp import dsl, compiler
from kfp.dsl import component, Input, Output, Dataset, Model, Metrics

PROJECT_ID = "mlops-484912"
REGION = "us-central1"
BUCKET_NAME = "meu-bucket-29061999"

# Inicializa Vertex AI
aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=f"gs://{BUCKET_NAME}"
)

@component(base_image="python:3.10-slim", packages_to_install=["pandas", "scikit-learn", "xgboost"])
def treinar_modelo(input_path: str, output_model: Output[Model], output_metrics: Output[Metrics]):
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from xgboost import XGBClassifier
    import pickle
    
    df = pd.read_csv(input_path)
    # ... (preprocessamento e treino)
    model = XGBClassifier()
    # model.fit(...)
    
    with open(output_model.path, "wb") as f:
        pickle.dump(model, f)

@dsl.pipeline(name="pipeline-inadimplencia")
def ml_pipeline(input_data: str):
    train_task = treinar_modelo(input_path=input_data)

# Compila e executa
compiler.Compiler().compile(ml_pipeline, "pipeline.json")
job = aiplatform.PipelineJob(
    display_name="inadimplencia-run",
    template_path="pipeline.json",
    pipeline_root=f"gs://{BUCKET_NAME}/pipeline_root",
)
job.submit()
'''

print("📋 Código do Pipeline Vertex AI:")
print("="*50)
print("Copie o código de jobs/vertex_pipeline.py e execute no GCP!")
print("\nOpções para executar:")
print("1. Cloud Shell: https://shell.cloud.google.com")
print("2. Vertex AI Workbench: https://console.cloud.google.com/vertex-ai/workbench")

## 📚 Resumo para Entrevista

### Ciclo Completo de MLOps:

```
1. DADOS → 2. PREPROCESSAMENTO → 3. TREINAMENTO → 4. AVALIAÇÃO → 5. REGISTRO → 6. DEPLOY → 7. MONITORAMENTO
     ↑                                                                                              |
     └──────────────────────────── FEEDBACK / RETREINO ─────────────────────────────────────────────┘
```

### Ferramentas que você usou:
- **MLflow**: Tracking de experimentos e Model Registry
- **Google Cloud Storage**: Armazenamento de dados e modelos
- **Vertex AI Pipelines**: Orquestração e automação
- **Kubeflow Pipelines (KFP)**: Definição de pipelines

### Perguntas comuns de entrevista:

1. **"Como você versiona modelos?"**
   - Resposta: MLflow Model Registry com tags de versão e stages (Staging, Production)

2. **"Como você monitora modelos em produção?"**
   - Resposta: Métricas de performance, data drift detection, logs estruturados

3. **"Como você faz deploy de um modelo?"**
   - Resposta: Batch scoring via pipeline ou endpoint REST para real-time

4. **"Como você garante reprodutibilidade?"**
   - Resposta: Pipelines versionados, containers Docker, tracking de experimentos

---
## 🎯 Próximos Passos

1. **Upload dos dados para o GCS:**
   ```bash
   gsutil cp data/base_clientes_inadimplencia.csv gs://meu-bucket-29061999/data/
   ```

2. **Executar o pipeline no Vertex AI:**
   ```bash
   python jobs/vertex_pipeline.py
   ```

3. **Visualizar no console:**
   - https://console.cloud.google.com/vertex-ai/pipelines

Boa sorte na entrevista! 🚀